# Getting data from aquamonitor

In [1]:
import requests
import getpass
import json
import xmltodict
from fabric import Connection

In [2]:
username = getpass.getpass('username: ')
password = getpass.getpass('password: ')

In [3]:
host = "http://www.aquamonitor.no/"

In [4]:
#Logging in
login_url =  host + "AquaServices" + "/WebServices/LoginService.asmx/AuthenticateUser"

r = requests.get(login_url,params=login_params)
display(r.status_code)
login_dict = xmltodict.parse(r.text)
token = login_dict['UserCargo']['Token']
key = login_dict['UserCargo']['Key']

headers = {'Cookie' : 'aqua_key={};'.format(token)}
headers

200

{'Cookie': 'aqua_key=MVXN8p9YjoJrYlQKvIS45KOvjjBU1NWQT4XmEJB15nk=;'}

In [5]:
project_codes = {'Langtjern' : 4300, 'Birkenes' : 2898} #The project should include the stations we are interested in

In [6]:
# Getting all data for available projects at Langtjern and Birkenes
#Filtering stations in project by name
station_codes = {}
for i,j in project_codes.items():
    projectsurl = 'AquaServices/api/projects/{projectId}/Stations'.format(projectId=j) #9906
    r = requests.get(host + projectsurl,headers=headers)
#     display(r.json())
    station_codes.update({k['Name'] : k['Id'] for k in r.json() if i in k['Name']})

station_codes

{'Bøye i Langtjern': 50472,
 'Utløp av Langtjern': 51356,
 'Utløp av Langtjern - jord': 51357,
 'Utløp av Langtjern - luft': 51358,
 'Værstasjon ved Langtjern': 62040,
 'Innløpet LAE03 - Langtjern': 63099,
 'Birkenes': 37927}

In [73]:
#Creating directory to store aquamonitor data
from fabric import Connection
with Connection('localhost') as c:
    c.local('''rm -rf aquamonitor && mkdir aquamonitor''')

In [76]:
#Creating data file
from datetime import date
archiveurl = host + 'AquaServices/files/archive/{aid}'
datafileurl = host + 'AquaServices/files/datafile'

today = date.today().strftime('%d.%m.%Y')
startDate = '01.01.1900' #So as to get the entire timeseries
# endDate = today

datawhere = '''sample_date>={startDate} and sample_date<={endDate}'''.format(startDate=startDate,
                                                                             endDate=today)

params = {"Expires": (date.today() + datetime.timedelta(days=1)).strftime('%Y.%m.%d'),
       "Title": '',
       "Files":[{"Filename": '',
                "ContentType":"text/csv"}],
       "Definition":{"Format":"csv",
                     "StationIds": [ '' ],
                     "DataWhere": datawhere
                    }
        }

headers['Content-Type'] = 'application/json'
display(headers)

for i,j in station_codes.items():
    params['Title'] = i.replace(' ','_')
    filename = params['Title'] + '.csv'
    params['Files'][0]['Filename'] = filename
    params['Definition']['StationIds'] = [j]
    display('Processing station {}'.format(i))
    #Create datafile in server
    r = requests.post(datafileurl,json=params,cookies=dict(aqua_key=token))
    currentId = r.json()['Id']
#     display(currentId)
    #Generating archive
    archived = False
    while not archived:
#         display(archiveurl.format(aid=currentId))
        resp = requests.get(archiveurl.format(aid=currentId),cookies = dict(aqua_key=token))
        archived = resp.json().get('Archived')
    #Downloading the archive
    fr = requests.get(archiveurl.format(aid=currentId) + '/' +filename ,cookies = dict(aqua_key=token))
    with open('./aquamonitor/{}'.format(filename),'wb') as fd:
        for chunk in fr.iter_content(chunk_size=256):
            fd.write(chunk)
        



{'Cookie': 'aqua_key=MVXN8p9YjoJrYlQKvIS45KOvjjBU1NWQT4XmEJB15nk=;',
 'Content-Type': 'application/json'}

'Processing station Bøye i Langtjern'

'zpdsvel'

'Processing station Utløp av Langtjern'

'rgumhmb'

...

'stdmuuz'

'Processing station Innløpet LAE03 - Langtjern'

'nttpqve'

'Processing station Birkenes'

'bzylsud'

In [77]:
import pandas as pd
df = pd.read_csv('./aquamonitor/Birkenes.csv',delimiter=';',encoding='utf-16')
df

,ProjectId,ProjectName,StationId,StationCode,StationName,SampleDate_dato,SampleDate_tid,Depth1,Depth2,Al/IL_µg/l,...,NH4-N_µg/l,NO3-N_µg/l,pH,PO4-P_µg/l,SO4_mg/L,TOC_mg/l,TOTN_µg/l,TOTN (est.)_µg/l,TOTN (old EF)_µg/l,TOTP_µg/l
0,2898,FELTFO,37927,BIE01,Birkenes,07.05.2018,0:00:00,0.0,0.0,100.0,...,< 2,250.0,4.71,< 1,1.56,5.6,450.0,NaN,NaN,3.0
1,2898,FELTFO,37927,BIE01,Birkenes,14.05.2018,0:00:00,0.0,0.0,100.0,...,< 2,110.0,5.11,< 1,1.56,6.4,NaN,361.670,280.0,4.0
2,2898,FELTFO,37927,BIE01,Birkenes,21.05.2018,0:00:00,0.0,0.0,96.0,...,< 2,100.0,4.94,< 1,1.69,5.9,NaN,351.693,270.0,4.0
3,2898,FELTFO,37927,BIE01,Birkenes,28.05.2018,0:00:00,0.0,0.0,100.0,...,4.0,73.0,5.04,< 1,1.63,6.7,NaN,349.093,260.0,5.0
4,2898,FELTFO,37927,BIE01,Birkenes,04.06.2018,0:00:00,0.0,0.0,140.0,...,7.0,33.0,5.03,< 1,1.52,11.5,NaN,524.406,360.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,12470,Sur nedbør - Feltfo 2019,37927,BIE01,Birkenes,01.02.2021,0:00:00,0.0,0.0,52.0,...,36.0,120.0,5.09,< 1,1.68,3.2,270.0,NaN,NaN,1.0
701,12470,Sur nedbør - Feltfo 2019,37927,BIE01,Birkenes,08.02.2021,0:00:00,0.0,0.0,43.0,...,48.0,150.0,5.19,< 1,2.03,2.9,250.0,NaN,NaN,2.0
702,12470,Sur nedbør - Feltfo 2019,37927,BIE01,Birkenes,15.02.2021,0:00:00,0.0,0.0,57.0,...,62.0,140.0,5.66,< 1,1.84,2.9,310.0,NaN,NaN,2.0
703,12470,Sur nedbør - Feltfo 2019,37927,BIE01,Birkenes,01.03.2021,0:00:00,0.0,0.0,65.0,...,9.0,150.0,4.83,< 1,1.52,3.6,290.0,NaN,NaN,2.0
